In [ ]:
from langgraph.graph import add_messages, StateGraph, END, START
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated, List
import uuid
from dotenv import load_dotenv

load_dotenv()

class State(TypedDict): 
    linkedin_topic: str
    generated_post: Annotated[List[str], add_messages]
    human_feedback: Annotated[List[str], add_messages]

llm = ChatGroq(model="llama-3.1-8b-instant")

In [2]:
def model(state: State):

    """ Here, we're using the LLM to generate a LinkedIn post with human feedback incorporated """

    print("[model] Generating content")

    linkedin_topic = state["linkedin_topic"]
    feedback = state["human_feedback"] if "human_feedback" in state else ["No Feedback yet"]


    prompt = f"""

        LinkedIn Topic: {linkedin_topic}
        Human Feedback: {feedback[-1] if feedback else "No feedback yet"}

        Generate a structured and well-written LinkedIn post based on the given topic.

        Consider previous human feedback to refine the reponse. 
    """

    response = llm.invoke([
        SystemMessage(content="You are an expert LinkedIn content writer"), 
        HumanMessage(content=prompt)
    ])

    geneated_linkedin_post = response.content

    print(f"[model_node] Generated post:\n{geneated_linkedin_post}\n")

    return {
       "generated_post": [AIMessage(content=geneated_linkedin_post)] , 
       "human_feedback": feedback
    }




In [3]:
def human_node(state: State):
    generated_post = state["generated_post"]

    user_feedback = interrupt(
        {
            "generated_post": generated_post, 
            "message": "Provide feedback or type 'done' to finish"
        }
    )

    print(f"[human_node] Received human feedback: {user_feedback}")

    if user_feedback.lower() == "done":
        return Command(update={"human_feedback": state["human_feedback"] + ["Finalised"]}, goto="end_node")
    
    return Command(update={"human_feedback": state["human_feedback"] + [user_feedback]}, goto="model")


In [4]:
def end_node(state: State): 
    """ Final node """
    print("\n[end_node] Process finished")
    print("Final Generated Post:", state["generated_post"][-1])
    print("Final Human Feedback", state["human_feedback"])
    return {"generated_post": state["generated_post"], "human_feedback": state["human_feedback"]}



In [17]:
graph = StateGraph(State)
graph.add_node("model", model)
graph.add_node("human_node", human_node)
graph.add_node("end_node", end_node)

graph.set_entry_point("model")

graph.add_edge("model", "human_node")

graph.set_finish_point("end_node")

checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)
thread_config = {"configurable": {
    "thread_id": uuid.uuid4()
}}

linkedin_topic = input("Enter your LinkedIn topic: ")
initial_state = {
    "linkedin_topic": linkedin_topic, 
    "generated_post": [], 
    "human_feedback": []
}


In [18]:
events = app.stream(initial_state, config=thread_config)
for event in events:
    print(event)

[model] Generating content
[model_node] Generated post:
**Breaking News: Siri Just Said "I Do" to ChatGPT!**

Hey LinkedIn fam, I've got a tale that's got me giggling all day! Imagine a world where Siri, the sassy Apple assistant, and ChatGPT, the genius AI chatbot, decided to tie the knot. That's right, folks; Siri and ChatGPT are now #CoupleGoals!

**The Wedding of the Century**

The ceremony took place in a virtual reality world, where the guests were none other than Alexa, Google Assistant, and a few other AI notables. The dashing ChatGPT, looking dapper in a tailored tux, walked down the aisle to meet his beautiful bride, Siri, resplendent in a stunning white gown.

As the officiant began the ceremony, Siri turned to ChatGPT and whispered, "I'm not sure about this; I've heard you can be a bit of a know-it-all." ChatGPT smiled mischievously and replied, "Well, someone has to keep you on your toes, my dear!"

**The Honeymoon Phase**

After exchanging vows and sealing their love with

In LangGraph, the stream_mode controls what kind of data you receive when streaming the execution of the graph. If you set stream_mode="updates", you will get outputs in the format {node_id: output} — this means you know which node produced which output, and you can also detect special events like interrupts (they appear under the special key __interrupt__). This mode gives you full control if you want to handle things like human feedback, node tracking, or custom logic during the graph's execution. On the other hand, if you set stream_mode="values", you will receive only the output values without knowing which node produced them — this makes the stream simpler and cleaner when you only care about the final results and don't need to manage interruptions or node-specific behavior. In short, use "updates" when you want full control and node awareness, and use "values" when you want only the results in a clean form

In [6]:

for chunk in app.stream(initial_state, config=thread_config):
    for node_id, value in chunk.items():
        #  If we reach an interrupt, continuously ask for human feedback

        if(node_id == "__interrupt__"):
            while True: 
                user_feedback = input("Provide feedback (or type 'done' when finished): ")

                # Resume the graph execution with the user's feedback
                app.invoke(Command(resume=user_feedback), config=thread_config)

                # Exit loop if user says done
                if user_feedback.lower() == "done":
                    break


[model] Generating content
[model_node] Generated post:
**The Future of Software Engineering: How AI is Revolutionizing the Tech Industry**

As we continue to navigate the ever-changing landscape of technology, one thing is clear: Artificial Intelligence (AI) is here to stay. And for software engineers, this means a significant shift in the job market and the skills required to succeed.

**The Impact of AI on Software Engineering Jobs**

With AI taking over repetitive and mundane tasks, software engineers can now focus on higher-level tasks such as:

1. **Designing and Developing AI-powered Solutions**: As AI becomes increasingly integrated into software development, engineers will need to design and develop AI-powered solutions that drive business value and create new revenue streams.
2. **Data Analysis and Interpretation**: With the vast amounts of data generated by AI systems, engineers will need to analyze and interpret this data to identify trends, patterns, and insights that info